# Hydrography
-----------------
This notebook explores the hydrographic data from the glider, with a focus on asepcts that are germane to ADCP processing. These include:
- Dive average current
- Geostrophic current
- Sound speed estimates
- Depth of pycnocline and other interfaces commonly associated with velocity shear
- Water mass identification

In [ ]:
import sys
import numpy as np
import pandas as pd
import xarray as xr
import copy
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
from netCDF4 import Dataset
from pathlib import Path
from datetime import datetime, timedelta
notebook_dir = !pwd # Replace with !echo %cd% if using windows
library_dir = Path(notebook_dir[0]).parent.absolute()
sys.path.append(str(library_dir))
data_raw_path = library_dir / 'data' / 'raw' / 'adcp'
glider_raw_path = library_dir / 'data' / 'raw' / 'glider'
fig_path = library_dir / 'reports' / 'figures'
from src.data.adcp_mission_analysis import read_glider_nc, edgetocentre

style_path = library_dir / 'src' / 'visualization' / 'presentation.mplstyle'
style.use(str(style_path))


def savefig(figname, extension="png"):
    plt.savefig(
        fig_path / str(figname + "." + extension),
        format=extension,
        dpi="figure",
        bbox_inches="tight",
    )



In [ ]:
glider_df = read_glider_nc(glider_raw_path / 'sg637_var_by_var_timeseries.nc')
#glider_df = read_glider_nc(glider_raw_path / 'sg637_flag_all_timeseries.nc')

In [ ]:
glider_df.salinity[glider_df.salinity<34.5] = np.nan
glider_df.salinity[glider_df.salinity>37.5] = np.nan
glider_df.abs_salinity[np.isnan(glider_df.salinity)] = np.nan
glider_df.conductivity[np.isnan(glider_df.salinity)] = np.nan

In [ ]:
np.nansum(np.isnan(glider_df.abs_salinity.values))

In [ ]:
origin  = [-57.338, 14.182]
deg_lat = 110649
glider_df['x'] = (glider_df.lon - origin[0]) * deg_lat * np.cos(np.deg2rad(origin[1]))
glider_df['y'] = (glider_df.lat - origin[1]) * deg_lat

In [ ]:
glider_av_df = glider_df.groupby('dive', as_index=False).median()

In [ ]:
glider_av_df.describe()

In [ ]:
glider_av_df['glider_time'] = ''
noot = []
for dive in glider_av_df.dive:
    df = glider_df[glider_df.dive==dive]
    noot.append((df.glider_time - df.glider_time.min()).mean() + df.glider_time.min())
glider_av_df['glider_time'] = noot

In [ ]:
glider_av_df

In [ ]:
box_size = 1000
lon_edges = np.arange(-8000, 9000, box_size)
lat_edges = np.arange(-8000, 9000, box_size)
dives_in = np.empty((len(lon_edges), len(lat_edges)))
dives_in[:] = np.nan
for i in range(len(lon_edges[:-1])):
    glider_x_in = glider_av_df[np.logical_and(glider_av_df.x>lon_edges[i], glider_av_df.x<lon_edges[i+1])]
    for j in range(len(lat_edges[:-1])):
        glider_in = glider_x_in[np.logical_and(glider_x_in.y>lat_edges[j], glider_x_in.y<lat_edges[j+1])]
        dives_in[j,i] = len(glider_in)

### Geographical location of glider dives

Here we plot the locations of the glider dives. This is the average of glider lon and lat for each dive, converted to distance from the central waypoint

In [ ]:
fig, ax = plt.subplots(figsize=[10,10])
ax.scatter(glider_av_df.x/1000, glider_av_df.y/1000)
ax.set(xlim=[-8, 8], ylim=[-8, 8], xlabel='Zonal distance from central waypoint (km)', ylabel='Meridional distance from central waypoint (km)')

As above for the density of glider dives in each 1 km square box

In [ ]:
fig, ax = plt.subplots(figsize=[12,10])
c = ax.pcolor(lon_edges/1000, lat_edges/1000,dives_in, cmap='Blues')
ax.set(xlabel='Zonal distance from center (km)', ylabel='Meridional distance from center (km)')
fig.colorbar(c, ax=ax, label='Number of dives in 1km square box')
savefig('Dive_concentration')


The maximum distance between two glider dives is 15 km. At 14 degrees north of the equator in a water depth of several km, the approximation of a virtual mooring seems appropriate. 

Plotting the variation of dive average current during the deployment. These are coloured by heading. 

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(16,14), sharex=True)
ax = ax.ravel()
ax[0].scatter(glider_av_df.glider_time, glider_av_df.DAC_u, c=glider_av_df.heading)
ax[1].scatter(glider_av_df.glider_time, glider_av_df.DAC_v, c=glider_av_df.heading)
ax[0].set(ylabel='DAC u $(\mathrm{m\ s^{-1}})$', ylim=[-0.2, 0.2])
ax[1].set(ylabel='DAC v $(\mathrm{m\ s^{-1}})$', xlim = [glider_av_df.glider_time[0]- timedelta(days=1), glider_av_df.glider_time.iloc[-1]+ timedelta(days=1)], ylim=[-0.2, 0.2])
savefig('DAC_by_dive_bearing')

Two patterns are apparent, the long term change in current speed and direction and a short term signal, more obvious in the u component, that shows a dependency on heading. This is a sign of a poorly calibrated compass

In [ ]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(x, y)
    deg = (180/np.pi)*phi
    if deg<0:
        deg = 360+deg
    return(rho, deg)
cart2pol=np.vectorize(cart2pol)

In [ ]:
glider_av_df['current_speed'], glider_av_df['current_bearing'] = cart2pol(glider_av_df.DAC_u, glider_av_df.DAC_v)

Plotting current speed and direction shows a switch around 5th February. There is nothing in the glider logs around this time. The compass calibration was performed on January 29.

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(16,14), sharex=True)
ax = ax.ravel()
ax[0].scatter(glider_av_df.glider_time, glider_av_df.current_speed, c=glider_av_df.heading, vmin=0, vmax=360)
scat = ax[1].scatter(glider_av_df.glider_time, glider_av_df.current_bearing, c=glider_av_df.heading, vmin=0, vmax=360)
ax[0].set(ylabel='Current speed $(\mathrm{m\ s^{-1}})$', ylim=[0, 0.3])
ax[1].set(ylabel='Current bearing $(\mathrm{m\ s^{-1}})$', xlim = [glider_av_df.glider_time[0]- timedelta(days=1), glider_av_df.glider_time.iloc[-1]+ timedelta(days=1)], ylim=[0,360])
sub_ax = plt.axes([0.62, 0.22, 0.2, 0.02])
fig.colorbar(mappable=scat, orientation='horizontal',cax=sub_ax, label='glider heading')
#savefig('current_heading')

In [ ]:
fig, ax = plt.subplots( figsize=(15,15))
ax.scatter(glider_df.abs_salinity, glider_df.temp, c=glider_df.glider_time, s = 1)

